In [1]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from IPython.display import display
from geopy.geocoders import Nominatim
from pymongo import MongoClient

In [ ]:
place = input ("Enter Address :") 
print(place)

In [ ]:
#place = "408 Brook Pine Trl, apex"
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(place) 
print(location)

In [ ]:
print((location.latitude, location.longitude))

In [ ]:
  # Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
  # Define the lat, long of the location and the year
  #lat, lon, year = 33.2164, -97.1292, 2018
  lat, lon =  location.latitude, location.longitude
  # You must request an NSRDB api key from the link above
  api_key = '5qyFRrBVjEZIGuR0WEcihqCEcg4LV8DbErgE6rze'
  # Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
  attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
  #attributes = 'ghi'
  # Choose year of data
  year = '2018'
  # Set leap year to true or false. True will return leap day data if present, false will not.
  leap_year = 'false'
  # Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
  interval = '60'
  # Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
  # NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
  # local time zone.
  utc = 'false'
  # Your full name, use '+' instead of spaces.
  your_name = 'Anthony+N'
  # Your reason for using the NSRDB.
  reason_for_use = 'testing'
  # Your affiliation
  your_affiliation = 'ECU'
  # Your email address
  your_email = 'natalea20@students.ecu.edu'
  # Please join our mailing list so we can keep you up-to-date on new developments.
  mailing_list = 'false'

  # Declare url string
  url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
  # Return just the first 2 lines to get metadata:
  info = pd.read_csv(url, nrows=1)
  # See metadata for specified properties, e.g., timezone and elevation
  timezone, elevation = info['Local Time Zone'], info['Elevation']
  
  df = pd.read_csv(url, nrows=2)
    

In [ ]:
# View metadata
info

In [ ]:
df = pd.read_csv(url, skiprows=2)
df.head()

In [ ]:
# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
df.shape

In [ ]:
df.head()

In [ ]:
# Print column names
df.columns.values

# Visualize the GHI Data for the Year 2018

In [ ]:
plt.plot(df.index, df['GHI'], alpha = 0.5, linewidth=.3)
plt.show()

As you can see, there are many ups and downs - these correspond to the time of day. Naturally, GHI is far lower during the night. The beginning and end of this graph is the evening and the center is when the sun is highest.

In [ ]:
zoomedin = plt.subplot()
zoomedin.margins(x=-0.498)
zoomedin.plot(df.index, df['GHI'])
zoomedin.set_title('Subset Plot')

In [ ]:
df.to_csv("sample.csv")
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client['Solar']
collection = db['Local']
df.reset_index(inplace=True)
data_dict = df.to_dict("records")
# Insert collection
collection.insert_many(data_dict)

# Now, Let's Run a Linear Regression Algorithm on the Data

## Get 3 Years of Data (2017-2019)

In [2]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
from IPython.display import display
from geopy.geocoders import Nominatim
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score,max_error
from sklearn import svm
from sklearn.pipeline import make_pipeline

In [3]:
place = input ("Enter Address :")

Enter Address :536 Garrison Ct SW Concord NC 28025


In [4]:
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(place)

years = ['2017','2018','2019']
df = []

lat, lon =  location.latitude, location.longitude
api_key = '5qyFRrBVjEZIGuR0WEcihqCEcg4LV8DbErgE6rze'
attributes = 'ghi'
leap_year = 'false'
interval = '60'
utc = 'false'
your_name = 'Anthony+N'
reason_for_use = 'testing'
your_affiliation = 'ECU'
your_email = 'natalea20@students.ecu.edu'
mailing_list = 'false'
for year in years:
    url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
    df.append(pd.read_csv(url, skiprows=2))

noon_values = pd.DataFrame(columns=['Year','Month','Day','Hour','Minute','GHI'])
# Append 2017 noon values
noon_2017 = df[0].loc[df[0]['Hour'] == 12]
noon_values = noon_values.append(noon_2017)
# Append 2018 noon values
noon_2018 = df[1].loc[df[1]['Hour'] == 12]
noon_values = noon_values.append(noon_2018)
# Append 2019 noon values
noon_2019 = df[2].loc[df[2]['Hour'] == 12]
noon_values = noon_values.append(noon_2019)

# Reset the index
noon_values = noon_values.reset_index()
# Eliminate the repetitive columns
# Leaves us with the datetime as the index and the GHI value at noon as the info
del noon_values['index']
del noon_values['Minute']
del noon_values['Hour']
noon_values['index1'] = noon_values.index

# Run the Linear Regression

In [16]:
# Get the features and GHI values
features = noon_values[['Year','Month','Day']]
results = noon_values['GHI']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, results, test_size=0.33)

# Scale the data
scaler = StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)


# Train the model on the scaled data
model = LinearRegression()
model.fit(X_scaled,y_train)
ghi_prediction = model.predict(X_test)

In [17]:
X_scaled

array([[ 0.01652047, -1.00884966, -0.28384944],
       [ 0.01652047,  1.3101995 , -0.17113999],
       [ 0.01652047, -0.71896852, -0.50926833],
       ...,
       [-1.19442962, -0.13920623,  0.73053557],
       [ 1.22747055,  0.44055606, -0.28384944],
       [ 1.22747055, -0.42908737,  1.74492058]])

In [24]:
lin_mse = mean_squared_error(y_test,ghi_prediction)
lin_r2 = r2_score(y_test,ghi_prediction)
lin_maxerror = max_error(y_test,ghi_prediction)
print("Mean Squared Error: ",lin_mse,"\nR-Squared Error: ",lin_r2,"\nMax Error: ",lin_maxerror)

Mean Squared Error:  1119820948.3648024 
R-Squared Error:  -14714.405602525712 
Max Error:  33896.83463992829


# Round 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, KFold, cross_val_predict
from pandas.tseries.holiday import USFederalHolidayCalendar
from scipy.stats import uniform as sp_rand
sns.set()

In [ ]:
# TODO: Use 10-Fold Cross Validation to select the best model
# Our K will be 10 for 10-fold validation
k = 10
# Linear Regression Cross Validation Scores & Predictions
linear = LinearRegression()
linear_cross_val_scores = cross_val_score(linear, X_train, y_train, cv=k)
linear_cross_val_predict = cross_val_predict(linear, X_train, y_train, cv=k)
print("Linear Regression Cross Val Scores: ", linear_cross_val_scores)

In [ ]:
kf = KFold(n_splits=k, random_state=None)
ridge_best_alpha_average = 0
lasso_best_alpha_average = 0
# Split the day characteristic data along the indices chosen by the KFold object, and for each split...
for train_index, test_index in kf.split(X_mod):
    # Get our day characteristic training and test data
    X_train, X_test = X_mod[train_index], X_mod[test_index]
    # Get our total number of cyclists training and test data
    y_train, y_test = y_mod[train_index], y_mod[test_index]
    # Reshape the y_train data a bit
    # y_train = y_train.reshape(-1, 1)
    # X_train = X_train.reshape(-1, 1)
    # X_test = X_test.reshape(-1, 1)
    # y_test = y_test.reshape(-1, 1)

    # TODO: Use RandomizedSearchCV to tune the alpha parameter for Ridge
    alpha_grid = {"alpha": sp_rand()}
    rr = Ridge()
    ridge_rscv = RandomizedSearchCV(estimator=rr, param_distributions=alpha_grid, cv=2, n_iter=100)
    ridge_rscv.fit(X_train, y_train)
    # Record the best alpha parameter for ridge
    ridge_best_alpha = ridge_rscv.best_estimator_.alpha
    ridge_best_alpha_average += ridge_best_alpha

    # TODO: Use RandomizedSearchCV to tune the alpha parameter for Lasso
    lasso = Lasso(tol=0.01)
    lasso_rscv = RandomizedSearchCV(estimator=lasso, param_distributions=alpha_grid, cv=2, n_iter=100)
    lasso_rscv.fit(X_train, y_train)
    # Record the best alpha parameter for lasso
    lasso_best_alpha = lasso_rscv.best_estimator_.alpha
    lasso_best_alpha_average += lasso_best_alpha

ridge_best_alpha_average /= 10
lasso_best_alpha_average /= 10

print("Ridge Best Alpha: ", ridge_best_alpha_average)
print("Lasso Best Alpha: ", lasso_best_alpha_average)

# Lasso
lasso = Lasso(alpha=lasso_best_alpha_average, tol=0.01)
lasso_cross_val_scores = cross_val_score(lasso, X_mod, y_mod, cv=k)
lasso_cross_val_predict = cross_val_predict(lasso, X_mod, y_mod, cv=k)
print("Lasso Cross Val Scores: ", lasso_cross_val_scores)

# Ridge
ridge = Ridge(alpha=ridge_best_alpha_average)
ridge_cross_val_scores = cross_val_score(ridge, X_mod, y_mod, cv=k)
ridge_cross_val_predict = cross_val_predict(ridge, X_mod, y_mod, cv=k)
print("Ridge Cross Val Scores: ", ridge_cross_val_scores)

# Plot the linear regression, ridge, and lasso fit lines against the data
plt.scatter(X_mod[:, 8], y_mod, label="Actual Data", color="y", alpha=.7)
plt.plot(X_mod[:, 8], linear_cross_val_predict, label="Linear Regression", color="b", linewidth=1)
# Plot both the ridge regression line and the test data (as points)
plt.plot(X_mod[:, 8], ridge_cross_val_predict, label="Ridge Regression", color="g", linewidth=1)
# Plot both the lasso regression line and the test data (as points)
plt.plot(X_mod[:, 8], lasso_cross_val_predict, label="Lasso Regression", color="r", linewidth=1)
plt.legend()
plt.ylabel("Cyclists")
plt.xlabel("Hours of Daylight")
plt.show()

print("Mean Squared Error: ")
print("Mean Squared Error: ", mean_squared_error(y_mod, linear_cross_val_predict))
print("Mean Squared Error: ",mean_squared_error(y_mod, ridge_cross_val_predict))
print("Mean Squared Error: ",mean_squared_error(y_mod, lasso_cross_val_predict))

print("R^2 Score: ")
print("Linear Regression: ", r2_score(y_mod, linear_cross_val_predict, multioutput='uniform_average'))
print("Linear Regression: ", r2_score(y_mod, linear_cross_val_predict, multioutput='uniform_average'))
print("Linear Regression: ", r2_score(y_mod, linear_cross_val_predict, multioutput='uniform_average'))

